# Artificial Neural Network

In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
# from sklearn import metrics
from common import *
# import tarfile

In [2]:
if not os.path.exists('data.csv'):
    if not os.path.exists('Fake.csv') and not os.path.exists('True.csv'):
        download_data()
    label_data('Fake.csv', 'fake_labeled.csv', 1)
    label_data('True.csv', 'true_labeled.csv', 0)
    os.system("awk '(NR == 1) || (FNR > 1)' *labeled.csv > data.csv")

In [3]:
df = pd.read_csv('data.csv', usecols=['text', 'label'], encoding='utf-8')
df = df.sample(n=NUM_SAMPLES, random_state=RANDOM_SEED)
df.head()

,text,label
4528,Donald Trump is calling for one of the most co...,1
31727,WASHINGTON (Reuters) - Former Republican U.S. ...,0
10937,You re never to young to commit jihad Teachers...,1
13470,Laura Ingraham reminds the Never Trump people ...,1
40814,BERLIN/HANOVER (Reuters) - Germany s Social De...,0


In [4]:
countVectorizer = CountVectorizer()
embeddings = preprocess_data(df['text'], countVectorizer)

X_tr, X_ts, y_tr, y_ts= train_test_split(embeddings, df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

X_tr = X_tr.toarray()
X_ts = X_ts.toarray()

feature_size = X_tr.shape[1]

print(f'Train shape: {X_tr.shape}')
print(f'Test shape: {X_ts.shape}')

Train shape: (1500, 49912)
Test shape: (1500, 49912)


In [5]:
#first basic ann
from keras.models import Sequential
from keras import layers

input_dim = X_tr.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

2022-04-20 14:22:09.156060: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 14:22:09.156298: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                499130    
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 499,141
Trainable params: 499,141
Non-trainable params: 0
_________________________________________________________________


2022-04-20 14:22:11.416511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-20 14:22:11.416542: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-20 14:22:11.416565: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-04-20 14:22:11.416856: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
history = model.fit(
    X_tr, y_tr,
    epochs=EPOCHS,
    verbose=True,
    validation_data=(X_ts, y_ts),
    batch_size=10
)

2022-04-20 14:22:16.855351: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 598944000 exceeds 10% of free system memory.


Epoch 1/100
144/150 [===========================>..] - ETA: 0s - loss: 0.2457 - accuracy: 0.9382

2022-04-20 14:22:19.846404: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 598944000 exceeds 10% of free system memory.


150/150 [==============================] - 3s 19ms/step - loss: 0.2434 - accuracy: 0.9373 - val_loss: 0.1518 - val_accuracy: 0.9687
Epoch 2/100
150/150 [==============================] - 1s 10ms/step - loss: 0.0497 - accuracy: 0.9993 - val_loss: 0.1337 - val_accuracy: 0.9727
Epoch 3/100
150/150 [==============================] - 1s 10ms/step - loss: 0.0277 - accuracy: 1.0000 - val_loss: 0.1287 - val_accuracy: 0.9720
Epoch 4/100
150/150 [==============================] - 1s 10ms/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 0.1280 - val_accuracy: 0.9713
Epoch 5/100
150/150 [==============================] - 1s 10ms/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.1294 - val_accuracy: 0.9720
Epoch 6/100
150/150 [==============================] - 2s 11ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.1328 - val_accuracy: 0.9707
Epoch 7/100
150/150 [==============================] - 2s 10ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.1365 - val_accuracy: 0.9713
Epoc

In [7]:
from keras.backend import clear_session
clear_session()

loss, accuracy = model.evaluate(X_tr, y_tr, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_ts, y_ts, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

import matplotlib.pyplot as plt
plt.style.use('ggplot')

2022-04-20 14:26:03.058681: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 598944000 exceeds 10% of free system memory.


Training Accuracy: 1.0000


2022-04-20 14:26:05.645248: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 598944000 exceeds 10% of free system memory.


Testing Accuracy:  0.9600


# CNN

In [ ]:
sentences_train, sentences_test, y_tr, y_ts= train_test_split(df['text'], df['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)
print(sentences_train)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_tr = tokenizer.texts_to_sequences(sentences_train)
X_ts = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_tr = pad_sequences(X_tr, padding='post', maxlen=maxlen)
X_ts = pad_sequences(X_ts, padding='post', maxlen=maxlen)

print(X_tr[0, :])


In [ ]:
embedding_dim = 50

model_cnn = Sequential()
model_cnn.add(Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model_cnn.add(layers.GlobalMaxPool1D())
model_cnn.add(layers.Dense(10, activation='relu'))
model_cnn.add(layers.Dense(1, activation='sigmoid'))
model_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_cnn.summary()

In [ ]:
model_cnn.fit(X_tr, y_tr, validation_data=(X_ts, y_ts), epochs=EPOCHS, batch_size=10, verbose=1)

In [ ]:
loss, accuracy = model_cnn.evaluate(X_tr, y_tr, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_cnn.evaluate(X_ts, y_ts, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

# BNN

In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from typing import *
# from sentence_transformers import SentenceTransformer

Prior and posterior likelihoods modeled by a zero-mean, unit-variance Gaussian normal distribution and multivariate Gaussian distribution with learnable mean and variances, respectively.

In [9]:
# zero mean, unit variance multivariate normal
def prior(kernel_size, bias_size, dtype=None):
        n = kernel_size + bias_size
        prior_model = keras.Sequential(
            [
                tfp.layers.DistributionLambda(
                    lambda t: tfp.distributions.MultivariateNormalDiag(
                        loc=tf.zeros(n), scale_diag=tf.ones(n)
                    )
                )
            ]
        )
        return prior_model

# variational multivariate normal (learnable means and variances)
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n),
                dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n)
        ]
    )
    return posterior_model

Create and compile model. The code below creates a hybrid BNN with 2 8-unit deterministic layers and a 2-unit probabilistic layer. The output is a normal distribution based on the parameters learned in the posterior functions/distribution.

In [10]:
inputs = keras.Input(
    shape=(feature_size,), dtype=tf.float64
)

features = keras.layers.BatchNormalization()(inputs)
features = keras.layers.Dense(8, activation='sigmoid')(features)
features = keras.layers.Dense(8, activation='sigmoid')(features)

distribution_params = tfp.layers.DenseVariational(
    units=2,
    make_prior_fn=prior,
    make_posterior_fn=posterior,
    kl_weight=1/TRAIN_SIZE
)(features)

outputs = tfp.layers.IndependentNormal(1)(distribution_params)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()]
)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


2022-04-20 14:26:19.421668: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Train and evaluate the model on the training and testing data.

In [11]:
model.fit(
    X_tr,
    y_tr,
    epochs=EPOCHS,
    verbose=1,
    validation_data=tuple([X_ts, y_ts])
)

Epoch 1/100


2022-04-20 14:26:22.553670: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 598944000 exceeds 10% of free system memory.


47/47 [==============================] - 5s 70ms/step - loss: 0.0014 - categorical_accuracy: 1.0000 - val_loss: 9.2589e-04 - val_categorical_accuracy: 1.0000
Epoch 2/100
47/47 [==============================] - 2s 46ms/step - loss: 8.3295e-04 - categorical_accuracy: 1.0000 - val_loss: 8.6331e-04 - val_categorical_accuracy: 1.0000
Epoch 3/100
47/47 [==============================] - 2s 45ms/step - loss: 8.9026e-04 - categorical_accuracy: 1.0000 - val_loss: 8.5515e-04 - val_categorical_accuracy: 1.0000
Epoch 4/100
47/47 [==============================] - 2s 46ms/step - loss: 9.0995e-04 - categorical_accuracy: 1.0000 - val_loss: 7.0427e-04 - val_categorical_accuracy: 1.0000
Epoch 5/100
47/47 [==============================] - 2s 45ms/step - loss: 8.0133e-04 - categorical_accuracy: 1.0000 - val_loss: 6.6651e-04 - val_categorical_accuracy: 1.0000
Epoch 6/100
47/47 [==============================] - 2s 53ms/step - loss: 6.7251e-04 - categorical_accuracy: 1.0000 - val_loss: 5.4418e-04 - val_c

In [12]:
clear_session()

loss, accuracy = model.evaluate(X_tr, y_tr)
print(f'Training accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_ts, y_ts)
print(f'Testing accuracy: {accuracy}')

47/47 [==============================] - 1s 24ms/step - loss: 8.7474e-05 - categorical_accuracy: 1.0000
Training accuracy: 1.0
47/47 [==============================] - 1s 15ms/step - loss: 1.0768e-04 - categorical_accuracy: 1.0000
Testing accuracy: 1.0


In [ ]:
PREDICTION_ITERATIONS = 100

results = pd.DataFrame(columns=['average', 'std', 'label'])

# predict on test samples
for i, sample in enumerate(X_ts):
    sample = sample.reshape(1, sample.shape[0])
    predictions = []

    for _ in range(PREDICTION_ITERATIONS):
        predictions.append(model.predict(sample))

    predictions = np.concatenate(predictions, axis=1)
    average = np.average(predictions, axis=1)
    standard_deviation = np.std(predictions, axis=1)

    frame = pd.DataFrame({'average': average, 'std': standard_deviation, 'label': y_ts.array[i]})
    pd.concat([results, frame])

    # print(
    #     f'Sample: {i}\n'
    #     f'Average: {average[0]}\n'
    #     f'Standard Deviation: {standard_deviation[0]}\n'
    #     f'Label: {y_ts.array[i]}\n\n'
    # )
results.head()